## Let's Code!

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import csv
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

2024-11-06 23:21:32.105112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 23:21:33.483522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 30000
MAX_LEN = 512
EMBEDDING_DIM = 512
KEY_DIM = 512
N_HEADS = 8
FEED_FORWARD_DIM = 512
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

## 1. Load the data <a name="load"></a>

In [3]:
# Load the full dataset
data_path = "/home/dsu/Desktop/Projects/LAB/Lab2/Transformer/movie_plots.csv"
movie_data = pd.read_csv(data_path)

In [4]:
movie_data.head()

,Unnamed: 0,title,plot
0,0,Animal Farm,"Old Major, the old boar on the Manor Farm, sum..."
1,1,A Clockwork Orange (novel),Alex is a 15-year-old living in near-future dy...
2,2,The Plague,The text of The Plague is divided into five pa...
3,3,Actaeon,"Among others, John Heath has observed, ""The un..."
4,4,A Fire Upon the Deep,"An expedition from Straumli Realm, an ambitiou..."


In [5]:
# Remove useless column
movie_data.drop("Unnamed: 0", axis=1, inplace=True)
movie_data.head()

,title,plot
0,Animal Farm,"Old Major, the old boar on the Manor Farm, sum..."
1,A Clockwork Orange (novel),Alex is a 15-year-old living in near-future dy...
2,The Plague,The text of The Plague is divided into five pa...
3,Actaeon,"Among others, John Heath has observed, ""The un..."
4,A Fire Upon the Deep,"An expedition from Straumli Realm, an ambitiou..."


In [6]:
# Look at the 11th entry
movie_data.iloc[10]["plot"]

'In Los Angeles in November 2019, ex-police officer Rick Deckard is detained by officer Gaff and brought to his former supervisor, Bryant.\nDeckard, whose job as a "Blade Runner" was to track down bioengineered beings known as replicants and "retire" (a euphemism for killing) them, is informed that four have come to Earth illegally.\nAs Tyrell Corporation Nexus-6 models, they have only a four-year lifespan and may have come to Earth to try to extend their lives.\nDeckard watches a video of a Blade Runner named Holden administering the "Voight-Kampff" test designed to distinguish replicants from humans based on their emotional response to questions.\nThe test subject, Leon, shoots Holden after Holden asks about Leon\'s mother.\nBryant wants Deckard to retire Leon and the other three replicants: Roy Batty, Zhora, and Pris.\nDeckard initially refuses, but after Bryant ambiguously threatens him, he reluctantly agrees.\nDeckard begins his investigation at the Tyrell Corporation to ensure th

In [7]:
# Filter the dataset
filtered_data = []

for index, row in movie_data.iterrows():
    filtered_data.append(
        "movie review : "
        + str(row["title"])
        + " : "
        + str(row["plot"]))

In [8]:
# Count the movies
n_movies = len(filtered_data)
print(f'{n_movies} reviews loaded')


112936 reviews loaded


In [10]:
# Look at the filtered data

example = filtered_data[25]
print(example)


movie review : Bubblegum Crisis : The series begins in the late 2032, seven years after the Second Great Kanto Earthquake has split Tokyo geographically and culturally in two.
During the first episode, disparities in wealth are shown to be more pronounced than in previous periods in post-war Japan.
The main antagonist is Genom, a megacorporation with immense power and global influence.
Its main product are boomers—artificial cybernetic life forms that are usually in the form of humans with most of their body's being a machine; also known as cyberoids.
While Boomers are intended to serve mankind, they become deadly instruments in the hands of ruthless individuals.
The AD Police are tasked to deal with Boomer-related crimes.
One of the series' themes is the inability of the department to deal with threats due to political infighting, red tape, and an insufficient budget.


## 2. Tokenize the data <a name="tokenize"></a>

In [11]:
# Pad the punctuation, to treat them as separate 'words'

def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(' +', ' ', s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

In [12]:
# Display an example of the padded data

example_data = text_data[25]
print(example_data)

movie review : Bubblegum Crisis : The series begins in the late 2032 , seven years after the Second Great Kanto Earthquake has split Tokyo geographically and culturally in two . 
 During the first episode , disparities in wealth are shown to be more pronounced than in previous periods in post - war Japan . 
 The main antagonist is Genom , a megacorporation with immense power and global influence . 
 Its main product are boomers—artificial cybernetic life forms that are usually in the form of humans with most of their body ' s being a machine ; also known as cyberoids . 
 While Boomers are intended to serve mankind , they become deadly instruments in the hands of ruthless individuals . 
 The AD Police are tasked to deal with Boomer - related crimes . 
 One of the series ' themes is the inability of the department to deal with threats due to political infighting , red tape , and an insufficient budget . 


In [13]:
# Find average length, used for figuring out a good context window
count = 0.0
for i in range(len(text_data)):
    count += len(text_data[i])

print(count / n_movies)
# Decided to use a context size of 512, it feels like it should be about right for this

2193.1180668697316


In [14]:
# Convert to a Tensorflow Dataset

text_ds = (tf.data.Dataset.from_tensor_slices(text_data).batch(BATCH_SIZE).shuffle(1000))

2024-11-06 23:22:53.670343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-06 23:22:53.783446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-06 23:22:53.786728: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
# Create a vectorization layer

vectorize_layer = layers.TextVectorization(standardize='lower', max_tokens=VOCAB_SIZE,
                                           output_mode='int', 
                                           output_sequence_length=MAX_LEN + 1)

In [16]:
# Adapt the layer to the training set

vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

2024-11-06 23:23:41.549531: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
# Display some token:word mappings

for i, word in enumerate(vocab[:10]):
    print(f'{i}: {word}')

0: 
1: [UNK]
2: the
3: ,
4: .
5: to
6: and
7: a
8: of
9: in


In [18]:
# Display the same example converted to ints

example_tokenized = vectorize_layer(example_data)
print(example_tokenized.numpy())


[   42    45    27 13937  2568    27     2   260   128     9     2   530
     1     3   915   120    40     2   319   374 29757  4964    34  2311
  2534     1     6     1     9    62     4   101     2    92   588     3
     1     9  2110    38   746     5    43   104  9041   212     9   686
  8661     9  1273    26   184  1626     4     2   600  6366    10     1
     3     7     1    17  6776   283     6  4497  1779     4   169   600
  5772    38     1 13669    74  1761    14    38  3009     9     2   504
     8   709    17   284     8    37   216    11    16    80     7   729
    65    70   330    23     1     4    56     1    38  2130     5  2339
  4580     3    29   177  2341  8348     9     2   540     8  3591  4613
     4     2  3742   119    38  4833     5   585    17 13725    26  2818
  1967     4    50     8     2   260    11  6262    10     2  5213     8
     2  1459     5   585    17  4156   280     5  1084     1     3   450
  1989     3     6    35 17941  8220     4     0   

## 3. Create the Training Set <a name="create"></a>

In [19]:
# Create the training set of recipes and the same text shifted by one word

def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]

    return x, y

train_ds = text_ds.map(prepare_inputs)

In [20]:
# Look at an example of an input/output sequence

example_input_output = train_ds.take(1).get_single_element()


In [21]:
# Example Input

example_input_output[0][0]

<tf.Tensor: shape=(512,), dtype=int64, numpy=
array([   42,    45,    27,  3808,   223,  2683,    27,     1,    34,
        1868,  3808,     6,    58,     8,    12,   144,     5,    96,
          28,     7,  5812,  4687,   204,     2,   126,   280,     5,
          58,     2,   853,   176,    65,    84,     3, 10037,    41,
          49,  1868,     4,  1438,    33,    80, 22432,     3,     2,
           1,   224,  1847,   496,     4,    39,     2,  4687,   227,
         190,    33,   169,    92,  3764,     3,     2,  2253,   357,
          14, 10037,    34,   839,    37,  2183,  4316,    47,     7,
        4599,    26,  1514,     1,     4,  3808,   121,     5,  1359,
          23,   235,  2117,    23,  1043,     5,   172,    52,     4,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   

In [22]:
# Example Output (shifted by one token)

example_input_output[1][0]

<tf.Tensor: shape=(512,), dtype=int64, numpy=
array([   45,    27,  3808,   223,  2683,    27,     1,    34,  1868,
        3808,     6,    58,     8,    12,   144,     5,    96,    28,
           7,  5812,  4687,   204,     2,   126,   280,     5,    58,
           2,   853,   176,    65,    84,     3, 10037,    41,    49,
        1868,     4,  1438,    33,    80, 22432,     3,     2,     1,
         224,  1847,   496,     4,    39,     2,  4687,   227,   190,
          33,   169,    92,  3764,     3,     2,  2253,   357,    14,
       10037,    34,   839,    37,  2183,  4316,    47,     7,  4599,
          26,  1514,     1,     4,  3808,   121,     5,  1359,    23,
         235,  2117,    23,  1043,     5,   172,    52,     4,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   

## 5. Create the causal attention mask function <a name="causal"></a>

In [23]:
# Create the causal mask, needed for decoder transformers
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >=j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0)

    return tf.tile(mask, mult)


## 6. Create a Transformer Block layer <a name="transformer"></a>

In [24]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    # call method for the transformer block, where all the magic happens
    
    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output, attention_scores = self.attn(inputs, inputs, 
                                                       attention_mask=causal_mask,
                                                       return_attention_scores=True)
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)

        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## 7. Create the Token and Position Embedding <a name="embedder"></a>

In [25]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        ) # token embedding layer
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim) # token position embedding layer

    # The embedding is how we keep track of the tokens and their position

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## 8. Build the Transformer model <a name="transformer_decoder"></a>

In [26]:
# Put all the pieces together
inputs = layers.Input(shape=(None, ), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)

gpt = models.Model(inputs = inputs, outputs=[outputs, attention_scores])
gpt.compile('adam', loss=[losses.SparseCategoricalCrossentropy(), None])


In [27]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 512)      │    15,622,144 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 512),    │     8,928,768 │
│ (TransformerBlock)              │ (None, 8, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 30000)    │    15,390,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,940,912 (152.36 MB)

 Trainable params: 39,940,912 (152.36 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    gpt = models.load_model("./models/gpt", compile=True)

## 9. Train the Transformer <a name="train"></a>

In [38]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("movie review", max_tokens=MAX_LEN, temperature=1.0)

In [30]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [31]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5


I0000 00:00:1730957029.890753  412400 service.cc:145] XLA service 0x770f880042a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730957029.891931  412400 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-11-06 23:23:49.993781: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1730957030.192233  412400 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-11-06 23:23:50.547107: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1730957034.322132  412492 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1730

2581/3530 ━━━━━━━━━━━━━━━━━━━━ 2:16 144ms/step - loss: 3.6397

W0000 00:00:1730957462.550127  412397 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1730957465.742446  412846 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_33', 432 bytes spill stores, 400 bytes spill loads

I0000 00:00:1730957466.798942  412843 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_33', 200 bytes spill stores, 200 bytes spill loads

I0000 00:00:1730957467.002411  412844 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 672 bytes spill stores, 640 bytes spill loads

I0000 00:00:1730957467.571690  412843 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 100 bytes spill stores, 92 bytes spill loads

I0000 00:00:1730957467.662422  412844

3530/3530 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 3.4562

I0000 00:00:1730957653.388128  413096 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 328 bytes spill stores, 328 bytes spill loads

I0000 00:00:1730957655.128061  413096 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1730957660.258218  413176 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1730957663.062939  413175 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 332 bytes spill stores, 332 bytes spill loads

I0000 00:00:1730957665.278958  413259 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_7', 328 bytes spill stores, 328 bytes spill loads

I0000 00:00:1730957667.795538  413259


generated text:
movie review : my love ( film ) : this is a naive but rich man , who is orphaned is poor in [UNK] ( [UNK] ) . poor and is both hearted but due to his love for his sincere nature , taoist ways with what will happen to miss , especially wealthy , stockholm , invisible [UNK] ( sakthi ) [UNK] ( don [UNK] [UNK] ) , often battles with the girl manickam ( thulasi ) , [UNK] ( [UNK] ) , another noble ( [UNK] [UNK] ) . [UNK] ( [UNK] basu ) , threats , he and [UNK] , [UNK] love for [UNK] . [UNK] leave [UNK] in spite of his fighting , [UNK] was called up after a friend , his spirit is shown to be irish street so overwhelmed when she was treated more in the aftermath . once hari and at that time , their way to [UNK] , his family meets and figures : how to earn their lives , and doesn ' t lose their opportunity to work , is as their father’s funds . his wife begins to run away , one day after spending the day before . [UNK] finally understands that [UNK] has been living with their t

In [32]:
# Save the final model
gpt.save("./models/gpt.keras")

# 3. Generate text using the Transformer

In [33]:
# A function to allow us to see the probability scores and token attention

def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,' 
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            ) # try (0, 76, 153) if it's too light
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [41]:
info = text_generator.generate(
    'movie review', max_tokens=MAX_LEN, temperature=(1)
)


generated text:
movie review : full egypt song : in the 1950s , in siam , yeon yi manu [UNK] , the translator of [UNK] , is stretched to a higher point since [UNK] . despite his efforts , too one by nin , the japanese , is a cynical young man who suffers in his spine and asthma and his daughter - in - law is obsessed with bao ' s urge to have heaven , since being an athlete in one woman night , [UNK] his friend [UNK] cooper from the hong kong movie theatre . while playing a dangerous challenge : he attempts to try to catch up with prevent the imminent destruction of hurricane by the young woman . however , later , [UNK] by the chance to catch up with him and absent - minded ; ho wants to make sure get in to say her beloved ' s refusal to be in danger lest the shadow change his mind including beating up his own inability to use the request of his father ' s passing success away . [UNK] ' s [UNK] petition requires him to steal the power to reduce the sentence of violence . determined to

In [42]:
info = text_generator.generate(
    'movie review', max_tokens=MAX_LEN, temperature=(1)
)


generated text:
movie review : rain : heavy drinker ( wallace ford ) : a women ' s [UNK] . when a reporter strikes another reporter , a man . she runs to fame is the only consolation from simran , is the main writer of hollywood hills high school . wallace ' s girlfriend , and her fiance dale , takes his money and lessons . wallace will a foreign graduate student , continues his passion for opera in the forthcoming european sports court . wallace ' s apparent innocence becomes success , and the story ' s award - winning nba laurel ' s manipulative rival . wallace elects in his little - little sister eva winston , as a [UNK] by her boyfriend early and joins her , she learns that wallace , and madden ' s organizer norton , arrive on an unexpected occasion . the two form a program in the hopes that wallace would cure wallace ' s [UNK] . she is dismayed when wallace , wallace is shaken by the fact that wallace had sent ramu to an alcoholics anonymous meeting , and his friends to join wall

In [55]:
info = text_generator.generate(
    'movie review : the green apple', max_tokens=MAX_LEN, temperature=(1)
)


generated text:
movie review : the green apple ( 2016 film ) : restoring the revived leg less after her from the 70s cup in 1982 , after waking an argument with tino is thrown at him ( hans ) with a vhs tape recorder in the swimming cup . [UNK] also finds himself at a natural athlete ' s metal cup in his toilet paper , but first he cannot eat until the coffee cup victim wears brown [UNK] chips and roasted chicken with clothes , someone asks for directions before he can wake up from them . he tries to gas in the first episode , but later the solution is not explained . once within a mall , matching the soccer coach asks him to drive . after he is thrown , and the two sit at the table , watching the movie after his auto flips , but leaves him to contest the winner . the belcher children find his wallet which he went to attend . the local pta headed to [UNK] , memorize the admission , where he loses consciousness . there the others are treated poorly by the coach . zeke now owns a dairy 

In [58]:
info = text_generator.generate(
    'movie review : glass tower', max_tokens=MAX_LEN, temperature=(1)
)


generated text:
movie review : glass tower : it ' s anatomy program has many discussions surrounding aspects expand on different values of patients . and the book that become estranged from the evils of life " . there are part of worship and destruction . their relationship becomes everyday suddenly nightmares and hallucinations . 



In [59]:
info = text_generator.generate(
    'movie review : balloon nightmare', max_tokens=MAX_LEN, temperature=(1)
)


generated text:
movie review : balloon nightmare : after the final hours of the date of the arcade bombing day . beginning in the philippines and the entire week , mysteriously find themselves going to inhabit a world called no mess around the world and records their demo until they give him the snacks . they rely on each other online jobs to only fit in in seconds of trouble . two objectives suddenly disappear and thousands of years later similar and gut injuries . as they attempt to sneak into korea for horrors , they encounter the same contagion . the player decides that one day to potentially lethal levels of health . the next day [UNK] can begin reconnaissance while the [UNK] prepare to open a local restaurant . the game progresses quickly becomes as possible . as a consequence , link [UNK] imperfect memory continues , and they begin a relationship soon develops stronger feelings towards each other . additionally , the player has been manipulated by the other more complex than th

In [51]:
info = text_generator.generate(
    'movie review', max_tokens=MAX_LEN, temperature=(0.75)
)


generated text:
movie review : [UNK] : it is the story of a documentary crew . the 16 - year - old [UNK] , who is an 11 - year - old girl , who is an 18 - year - old girl . the family is not interested in him because she is sick and she has to face with her father and her eyes on his house . her father , however , her fears that she does not have a time of travel and he has to be with her . her father , she even meets her father - in - law , who likes her . she is not looking forward to the way she meets and falls in love with her . she becomes difficult to explain the situation to her father , who has recently become a renowned musician . she is still attracted to him , but she rejects him . she feels that she has an excuse herself to catch the boy . she is betrayed by her father . when she is out with her father , she refuses the offer to marry him . she decides to take revenge on her father . she is ready to live with the father . she is happy in his father ' s footsteps and marrie

In [52]:
info = text_generator.generate(
    'movie review', max_tokens=MAX_LEN, temperature=(0.5)
)


generated text:
movie review : the [UNK] : the film is about a young woman who lives in the woods with her parents , her parents and her parents . her parents are [UNK] and her parents , who are in a small village . the girls are taken to a house where they are treated like a family . they have a lot of problems and their parents . [UNK] and [UNK] are a couple of [UNK] and [UNK] . [UNK] mother and daughter go to college . they have a daughter , [UNK] is a girl who is a famous singer . [UNK] is a girl who dreams of becoming a singer . [UNK] is a singer . [UNK] is a singer and her father is a singer who is [UNK] ' s lover . [UNK] is a singer . [UNK] is a singer and [UNK] ' s father is a singer . [UNK] is a singer . [UNK] falls for [UNK] [UNK] . [UNK] ' s mother is a singer and [UNK] is the daughter of [UNK] . [UNK] is a singer and [UNK] is a singer . [UNK] is a singer and [UNK] is a singer but [UNK] is a singer . [UNK] is a singer , who is the singer of his daughter . [UNK] is the singe

In [60]:
info = text_generator.generate(
    'movie review : golden sunlight', max_tokens=MAX_LEN, temperature=(0.25)
)


generated text:
movie review : golden sunlight ( film ) : the film is set in the 1930s , the film centres around the life of a young woman who has been living in the 1950s . the film is set in the 1930s , and the film is set in the 1930s , and the film is set in the 1950s , and the film is set in the 1950s . the film is a [UNK] of the film , and the film is the film ' s main character , and the film is set in the film . the film is set in the film , the film is set in the film ' s [UNK] , and the film is set in the film ' s [UNK] . the film is set in the film , the film is set in the film ' s [UNK] , and the film is set in the film ' s [UNK] . 

